# Финальное обучение модели

Эксперименты и обзор методов расположены в ноутбуке solution.

In [1]:
import os

# Путь к корневой директории, если ноутбук запускался не из неё
PATH = "D:\\Documents\\GitHub\\medical-imaging-evaluation"
# Путь к папке с данными
DATA_PATH = "D:\\Documents\\GitHub\\medical-imaging-evaluation\\data"
os.chdir(PATH)

In [2]:
import cv2
import joblib
import numpy as np
import pandas as pd

from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor, VotingClassifier
from sklearn.svm import SVC

from model.metrics.metrics_wrapper import Metrics

### Метрики

Метрики реализованы на основе библиотеки https://github.com/deepmind/surface-distance. Помимо реализованных добавлены такие метрики, как Symmetric Surface Distance (средняя), Volume Overlap Error и Relative Volume Difference:

$$AvSD = \frac{1}{|X|+|Y|}\left(\sum D_Y(x)+\sum D_X(y)\right)$$

$$VOE = 100\times\left(1-2\frac{X\cap Y}{|X|+|Y|}\right)$$

$$RVD = \frac{2|X\cap Y|}{|X|+|Y|}$$

In [3]:
def compute_metrics(ex, pred):
    """
    Подсчёт значения метрик для двух изображений -- экспертной и оцениваемой разметок.
    Метрики: на основе расстояний, на основе объёма множеств и на основе расположения геометрических фигур (опционально).
    """
    result = []
    num_metrics = 12
    
    result.append(ex.sum() / 255)
    result.append(pred.sum() / 255)
    
    if ex.sum() == 0 and pred.sum() == 0:
        return result + [0] * num_metrics
    elif ex.sum() == 0 or pred.sum() == 0:
        return result + [np.NaN] * num_metrics

    metrics = Metrics(ex, pred)
    result = metrics.compute_all()
    
    return result

def make_dataframe(expert, predicted):
    """
    Создание pd.DataFrame из метрик, рассчитанных для каждой пары объектов из выборок expert и predicted.
    """
    data = []
    for i in range(len(expert)):
            data.append(np.array(compute_metrics(expert[i], predicted[i])))
    return pd.DataFrame(data)

### Обучение модели, сохранение и предсказание

Загрузка данных:

In [4]:
def load_data(names, folder, ending):
    """
    Загрузка необходимых изображений (имена в списке names) из указанной папки.
    """
    data = []
    for name in names:
        image = cv2.imread(folder + '/' + name.split('.')[0] + ending + '.png', 0)
        data.append(image)
    return np.array(data)

train_eval = pd.read_csv(DATA_PATH + '/OpenPart.csv')
test_eval = pd.read_csv(DATA_PATH + '/SecretPart_dummy.csv')

train_names = train_eval['Case'].to_numpy()
test_names = test_eval['Case'].to_numpy()

train_data = [
    load_data(train_names, DATA_PATH + '/sample_1', '_s1'),
    load_data(train_names, DATA_PATH + '/sample_2', '_s2'),
    load_data(train_names, DATA_PATH + '/sample_3', '_s3')
]

test_data = [
    load_data(test_names, DATA_PATH + '/sample_1', '_s1'),
    load_data(test_names, DATA_PATH + '/sample_2', '_s2'),
    load_data(test_names, DATA_PATH + '/sample_3', '_s3')
]

expert_train = load_data(train_names, DATA_PATH + '/expert', '_expert')
expert_test = load_data(test_names, DATA_PATH + '/expert', '_expert')

X = [
    make_dataframe(expert_train, train_data[0]),
    make_dataframe(expert_train, train_data[1]),
    make_dataframe(expert_train, train_data[2])
]

y = [
    train_eval['Sample 1'],
    train_eval['Sample 2'],
    train_eval['Sample 3']
]

for i in range(len(X)):
    X[i] = X[i].fillna(-1).to_numpy()

train_df = np.concatenate(X)

test_df = pd.concat([
    make_dataframe(expert_test, test_data[0]),
    make_dataframe(expert_test, test_data[1]),
    make_dataframe(expert_test, test_data[2])
]).fillna(-1).to_numpy()

scaler = StandardScaler()
train_df = scaler.fit_transform(train_df)
test_df = scaler.transform(test_df)

train_labels = np.concatenate(y)

Обёртка для регрессора с округлением:

In [5]:
class RoundClassifier(BaseEstimator, ClassifierMixin):
    """
    Округление предсказаний модели до целых чисел.
    """
    def __init__(self, model, round_model=True):
        self.model = model
        self.round_model = round_model
        
    def fit(self, X, y):
        self.model.fit(X, y)
        return self
    
    def predict(self, X):
        pred = self.model.predict(X)
        if self.round_model:
            pred = np.round(pred, 0).astype(int)
        return pred

Модель:

In [6]:
model = VotingClassifier(estimators=[('lr',LogisticRegression()), ('svc', SVC()),
                                     ('rf', RoundClassifier(RandomForestRegressor(random_state=0, criterion='mae')))])
model.fit(train_df, train_labels)

VotingClassifier(estimators=[('lr', LogisticRegression()), ('svc', SVC()),
                             ('rf',
                              RoundClassifier(model=RandomForestRegressor(criterion='mae',
                                                                          random_state=0)))])

Предсказания для тестовой выборки:

In [7]:
predictions = model.predict(test_df).reshape((3, 40))
test_eval['Sample 1'] = predictions[0]
test_eval['Sample 2'] = predictions[1]
test_eval['Sample 3'] = predictions[2]
test_eval

,Case,Sample 1,Sample 2,Sample 3
0,00011827_003.png,1,1,1
1,00011925_072.png,5,5,3
2,00012045_019.png,1,5,1
3,00012094_040.png,5,3,2
4,00012174_000.png,5,3,3
5,00012270_005.png,3,3,5
6,00012414_000.png,3,1,1
7,00012592_005.png,4,4,2
8,00012829_004.png,3,5,5
9,00012892_010.png,3,2,3


Сохранение модели и предсказаний:

In [8]:
joblib.dump(scaler, PATH + "\\model\\scaler_dump.pkl")

['D:\\Documents\\GitHub\\medical-imaging-evaluation\\model\\scaler_dump.pkl']

In [9]:
joblib.dump(model, PATH + "\\model\\model_dump.pkl")

['D:\\Documents\\GitHub\\medical-imaging-evaluation\\model\\model_dump.pkl']

In [10]:
test_eval.to_csv(PATH + "D:\Documents\Inspiration\\test_predictions.csv")